<a href="https://colab.research.google.com/github/felipemoreia/acelaradev-codenation/blob/master/teste_admissao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn import metrics


In [0]:
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv') 
df_resposta = pd.DataFrame()


In [0]:
train = train.select_dtypes(include=['int64','float64'])
df_resposta['NU_INSCRICAO'] = test['NU_INSCRICAO']

In [0]:
features = ['NU_NOTA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5','NU_NOTA_REDACAO']


In [0]:
features2 =  ['NU_INSCRICAO','NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']


In [0]:
dummieses = ['tp_escola_novo', 'tp_dependencia_novo']

train['tp_escola_novo'] = train['TP_ESCOLA'].replace([1,2,3,4],['nr', 'pub', 'pri', 'ext'])
train['tp_dependencia_novo'] = train['TP_DEPENDENCIA_ADM_ESC'].replace([1,2,3,4],['fed', 'est', 'mun', 'priv'])

dummies = pd.get_dummies(train[dummieses])
dummies[features] = train[features]

test['tp_escola_novo'] = test['TP_ESCOLA'].replace([1,2,3,4],['nr', 'pub', 'pri', 'ext'])
test['tp_dependencia_novo'] = test['TP_DEPENDENCIA_ADM_ESC'].replace([1,2,3,4],['fed', 'est', 'mun', 'priv'])

dummies_test = pd.get_dummies(train[dummieses])
dummies_test[features2] = test[features2]

In [0]:
dummies = dummies.loc[
      (dummies['NU_NOTA_CN'].notnull())  & (dummies['NU_NOTA_CN'] != 0) & (dummies['NU_NOTA_CH'].notnull())      & (dummies['NU_NOTA_CH'] != 0) 
    & (dummies['NU_NOTA_LC'].notnull())  & (dummies['NU_NOTA_LC'] != 0) & (dummies['NU_NOTA_REDACAO'].notnull()) & (dummies['NU_NOTA_REDACAO'] != 0)    
]

In [0]:
dummies_test = dummies_test.loc[
      (dummies_test['NU_NOTA_CN'].notnull())  & (dummies_test['NU_NOTA_CN'] != 0) & (dummies_test['NU_NOTA_CH'].notnull())      & (dummies_test['NU_NOTA_CH'] != 0) 
    & (dummies_test['NU_NOTA_LC'].notnull())  & (dummies_test['NU_NOTA_LC'] != 0) & (dummies_test['NU_NOTA_REDACAO'].notnull()) & (dummies_test['NU_NOTA_REDACAO'] != 0)    
]

In [0]:
dummies_test.isnull().sum()

tp_escola_novo_ext          0
tp_escola_novo_nr           0
tp_escola_novo_pri          0
tp_escola_novo_pub          0
tp_dependencia_novo_est     0
tp_dependencia_novo_fed     0
tp_dependencia_novo_mun     0
tp_dependencia_novo_priv    0
NU_INSCRICAO                0
NU_NOTA_CN                  0
NU_NOTA_CH                  0
NU_NOTA_LC                  0
NU_NOTA_COMP1               0
NU_NOTA_COMP2               0
NU_NOTA_COMP3               0
NU_NOTA_COMP4               0
NU_NOTA_COMP5               0
NU_NOTA_REDACAO             0
dtype: int64

In [0]:
y_train = dummies['NU_NOTA_MT']
X = dummies.drop('NU_NOTA_MT', axis=1)

In [0]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
df_scaler_test = dummies_test.drop('NU_INSCRICAO', axis=1)
X_test_scaled = scaler.fit_transform(df_scaler_test)
X_train_scaled = pd.DataFrame(X_train_scaled)


In [0]:
cv = 5
scoring = 'r2'
n_jobs = -1

In [0]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
scores = cross_val_score(model, X_train_scaled, y_train, cv = cv,
                         scoring = scoring, n_jobs = n_jobs)
scores.mean()

0.4394822601070715

In [0]:
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

In [0]:
dummies_test['NU_NOTA_MT'] = np.around(y_pred,2)

In [0]:
respostas = ['NU_NOTA_MT', 'NU_INSCRICAO']
dummies_resposta = dummies_test[respostas]

In [0]:
df_resposta = df_resposta.merge(dummies_resposta, on='NU_INSCRICAO', how='left')

In [0]:
df_resposta.head(50)

,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,436.02
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,448.63
2,b38a03232f43b11c9d0788abaf060f7366053b6d,584.28
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,NaN
4,715494628a50142ce8cb17191cfe6d0f3cae0934,542.62
5,e656d6bad65c93fb2880f1eba5037008c8e75774,474.13
6,465cd2a6907fb37d9d8ad3c065f0e2dabdba9b13,518.57
7,11539e86171bf07d3a36f09377d7f54ebcc8406a,438.05
8,043c544a2104aa8a9849f1a703a08d37a2f16839,474.11
9,76ba050e64ad100b856f0eaabd8f539d5c7dd185,455.78


In [0]:
df_resposta.to_csv('answer.csv', index=False, header=True)